In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import plotly as pl

In [ ]:
sys.path.insert(0, "..")

import ccal

np.random.random(20121020)

pl.offline.init_notebook_mode(connected=True)

In [ ]:
w = pd.read_table("w.tsv", index_col=0)

w.columns.name = "Factor"

h = pd.read_table("h.tsv", index_col=0)

h.columns.name = "Sample"

In [ ]:
gps_map = ccal.GPSMap(w=w.T, h=h)

In [ ]:
for w_or_h in ("w", "h"):

    gps_map.plot_gps_map(w_or_h)

In [ ]:
for w_or_h, elements in (("w", w.index), ("h", h.columns)):

    annotation_types = ("continuous", "categorical", "binary")

    annotation_x_element = pd.DataFrame(
        [
            np.random.random_sample(elements.size),
            np.random.randint(0, high=3, size=elements.size),
            np.random.randint(0, high=2, size=elements.size),
        ],
        index=annotation_types,
        columns=elements,
    )

    annotation_x_element.iloc[0, 0] = np.nan

    annotation_x_element.iloc[1, 1] = np.nan

    annotation_x_element.iloc[2, 2] = np.nan

    annotation_x_element.iloc[:, 3] = np.nan

    gps_map.plot_gps_map(
        w_or_h,
        annotation_x_element=annotation_x_element,
        annotation_types=annotation_types,
    )

    for annotation_type, annotation in annotation_x_element.iterrows():

        gps_map.plot_gps_map(
            w_or_h,
            annotation_x_element=annotation.to_frame().T,
            annotation_types=(annotation_type,),
            annotation_colorscale=ccal.make_colorscale(
                colors=("pink", "yellow", "purple"), plot=False
            ),
        )

    annotation_x_element = pd.DataFrame(
        [np.random.normal(size=elements.size)] * 3, columns=elements
    )

    gps_map.plot_gps_map(
        w_or_h,
        annotation_x_element=annotation_x_element,
        annotation_std_maxs=(0.1, 1, 3),
    )

    gps_map.plot_gps_map(
        w_or_h,
        annotation_x_element=annotation_x_element.iloc[:1],
        annotation_ranges=((0, 8),),
    )

In [ ]:
for w_or_h, element_labels in (
    ("w", w.apply(np.argmax, axis=1).str[1:].astype(int)),
    ("h", h.apply(np.argmax).str[1:].astype(int)),
):

    gps_map.set_element_labels(w_or_h, element_labels)

    gps_map.plot_gps_map(w_or_h)

In [ ]:
for w_or_h, w_or_h_matrix in (("w", w.T), ("h", h)):

    predicted_element_labels = gps_map.predict(w_or_h, w_or_h_matrix.iloc[:, :8])

    print(predicted_element_labels)

In [ ]:
for w_or_h in ("w", "h"):

    gps_map.anneal_node_and_element_positions(
        w_or_h, n_iteration=64, initial_temperature=0.01
    )

    gps_map.plot_gps_map(w_or_h)

In [ ]:
h = pd.read_table("kras_gps_map/nmf_k9_h.tsv", index_col=0)

# https://github.com/UCSD-CCAL/onco_gps_paper_analysis/blob/master/code/8%20Define%20global%20cellular%20labels%20and%20make%20global%20Onco-GPS%20map.ipynb

h = h.apply(lambda row: (row - row.mean()) / row.std(), axis=1)

h = h.clip(lower=-3, upper=3)

h = h.apply(lambda row: (row - row.min()) / (row.max() - row.min()), axis=1)

In [ ]:
# https://github.com/KwatME/gps_map/blob/e932c53e1d679a68c224cf62284fb9209235962f/onco_gps/GPSMap.py

h = h.apply(lambda row: (row - row.mean()) / row.std(), axis=1)

h = h.clip(lower=-3, upper=3)

h = h.apply(lambda row: (row - row.min()) / (row.max() - row.min()), axis=1)

# Euclidean distance is used instead in https://github.com/KwatME/gps_map/blob/e932c53e1d679a68c224cf62284fb9209235962f/onco_gps/GPSMap.py

gps_map = ccal.GPSMap(h=h, h_pull_power=2.4, plot=False)

In [ ]:
h_element_labels = pd.read_table("kras_gps_map/hcc__k_x_h_column.tsv", index_col=0).loc[
    "K15"
]

gps_map.set_element_labels(
    "h",
    h_element_labels,
    label_colors=(
        "#e74c3c",
        "#ffd700",
        "#4b0082",
        "#993300",
        "#4169e1",
        "#90ee90",
        "#f4bd60",
        "#8b008b",
        "#fa8072",
        "#b0e0e6",
        "#20d9ba",
        "#da70d6",
        "#d2691e",
        "#dc143c",
        "#2e8b57",
    ),
    bandwidth_factor=5.6,
)

gps_map.plot_gps_map("h")

In [ ]:
predicted_element_labels = gps_map.predict("h", h)

predicted_element_labels

In [ ]:
ccal.dump_gps_map(gps_map, "gps_map.pickle.gz")

In [ ]:
gps_map = ccal.load_gps_map("gps_map.pickle.gz")

gps_map.plot_gps_map("h")

os.remove("gps_map.pickle.gz")